In [4]:
import sys
import os
from midi2audio import FluidSynth
import pretty_midi
from os import listdir
import re
from google.cloud import storage
import pickle as pkl
import IPython.display as ipd
import random

In [5]:
sys.path.append('../progs')
from utility_functions import *

# Specify SoundFont for MIDI to WAV Conversion - Currently Default MuseScore

In [7]:
auth_json = '../gcp_bucket_auth/fyr-bucket-auth.json'
bucket_name = 'fyr-audio-data'

dirSFs = '/home/rcalzaretta_berkeley_edu/SFs/'
listSFs = []
listSFNames = []
#Gather Sound Font Files
for file in listdir(dirSFs):
    listSFs.append(file)
    listSFNames.append(file.split('.')[0])

print(listSFs)
print(listSFNames)
# create storage client
storage_client = storage.Client.from_service_account_json(auth_json)
# get bucket with name
bucket = storage_client.get_bucket(bucket_name)


['GoldDrums.sf2', '1115-Standard Rock Set.sf2', 'Tamburo_Formula.sf2', 'fUNKNewKit.sf2', 'MelottiDrums.sf2', '241-Drums.SF2', 'TamaRockSTAR.sf2', 'RealAcousticDrumsEXTRA.sf2', '1276-The KiKaZ DrUmZ.sf2', 'default_sound_font.sf2', 'PremierKit.sf2', '.ipynb_checkpoints', 'HardRockDrums.sf2', '566-HARD ROCK DRUMS V3.sf2', 'PNS Drum Kit.SF2']
['GoldDrums', '1115-Standard Rock Set', 'Tamburo_Formula', 'fUNKNewKit', 'MelottiDrums', '241-Drums', 'TamaRockSTAR', 'RealAcousticDrumsEXTRA', '1276-The KiKaZ DrUmZ', 'default_sound_font', 'PremierKit', '', 'HardRockDrums', '566-HARD ROCK DRUMS V3', 'PNS Drum Kit']


# Generate labels for eGMD data

In [16]:
# specify genre/groove type to pull
regexGenre = re.compile('rock|funk|soul')
regexSess = re.compile('session[0-9][0-9]?')
#if re.match(regex, folder):

# get a list of raw egmd midi files
storage_client = storage.Client.from_service_account_json(auth_json)
egmd_midi_files = [blob.name for blob in storage_client.list_blobs(bucket,prefix='raw-audio/eGMD/') 
                   if blob.name[-5:]=='.midi' and 
                   re.search(regexSess, blob.name) and re.search(regexGenre, blob.name)]
print(len(egmd_midi_files))

23650


In [18]:
#Randomize
print(egmd_midi_files[0])
random.shuffle(egmd_midi_files)
print(egmd_midi_files[0])

raw-audio/eGMD/e-gmd-v1.0.0-midi/drummer6/session3/7_rock_88_beat_4-4_28.midi
raw-audio/eGMD/e-gmd-v1.0.0-midi/drummer7/session2/24_rock_118_beat_4-4_37.midi


## Warning - the below cell takes a LONG (~60 hours) time to process all ~45k midi files (less if you don't do sr 9000 degraded files)

In [19]:
start_i = 0
end_i = 2
i = start_i

In [20]:
dictEGMD = {}
labels = []
#i = 0 # song index number

for file in egmd_midi_files[start_i:end_i]:
    drummer = file.split('/')[-3]
    subdir = file.split('/')[-2]
    song_name = file.split('/')[-1]
    
    # midi2labels imported from utility_functions
    l = midi2labels(auth_json=auth_json, bucket=bucket_name, gcp_filepath=file, alt=True)
    fname = drummer + '_' + subdir + '_' + song_name
    
    # update eGMD dict and labels list
    dictEGMD[i] = fname
    l = [i, l]
    labels.append(l)
    i += 1

    # read in the midi file from GCP as blob
    blob = bucket.get_blob(file)
    blob.download_to_filename('tempFile.midi')
    
    k = 0
    for SF in listSFs:
        fs = FluidSynth('/home/rcalzaretta_berkeley_edu/SFs/' + SF)
        fs9000 = FluidSynth('/home/rcalzaretta_berkeley_edu/SFs/' + SF, sample_rate=9000)
        SFName = listSFNames[k]
        print(SFName)
    
        # use FluidSynth with SondFont file to convert midi to wav
        fs.midi_to_audio('tempFile.midi', 'tempFile.wav')
        # small sample rate for purposefully degraded audio
        fs9000.midi_to_audio('tempFile.midi','tempFileDegraded.wav')
        # use gsutil to write both wav files to GCP bucket
        !gsutil -m cp tempFile.wav gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/{SFName}/{fname + '.wav'}
        !gsutil -m cp tempFileDegraded.wav gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/sr9000/{SFName}/{fname + '.wav'}
        os.remove('tempFile.wav')
        os.remove('tempFileDegraded.wav')
        k += 1
        
    os.remove('tempFile.midi')
#ipd.Audio('tempFile.wav')

GoldDrums
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  2.8 MiB/  2.8 MiB] 100% Done                                    
Operation completed over 1 objects/2.8 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][593.0 KiB/593.0 KiB] 100% Done                                    
Operation completed over 1 objects/593.0 KiB.                                    
1115-Standard Rock Set
CommandException: No URLs matched: gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/1115-Standard
CommandException: No URLs matched: Rock
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: 3 files/objects could not be transferred.
CommandException: No URLs matched: gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/sr9000/1115-Standard
CommandException: No URLs matched: Rock
CommandException: 

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][ 74.3 KiB/ 74.3 KiB] 100% Done                                    
Operation completed over 1 objects/74.3 KiB.                                     
1115-Standard Rock Set
CommandException: No URLs matched: gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/1115-Standard
CommandException: No URLs matched: Rock
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: 3 files/objects could not be transferred.
CommandException: No URLs matched: gs://fyr-audio-data/raw-audio/eGMD/eGMD-SoundFont-wavfiles/sr9000/1115-Standard
CommandException: No URLs matched: Rock
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: 3 files/objects could not be transferred.
Tamburo_Formula
Copying file://tempFile.wav [Content-

In [12]:
with open('eGMDdictSoundFonts.pkl','wb') as f:
    pkl.dump(dictEGMD, f, pkl.HIGHEST_PROTOCOL)
    
with open('eGMDlabelsSoundFonts.pkl','wb') as f:
    pkl.dump(labels, f, pkl.HIGHEST_PROTOCOL)
    
# use gsutil to write both wav files to GCP bucket
!gsutil -m cp eGMDdictSoundFonts.pkl gs://fyr-audio-data/raw-audio/SoundFontFiles/eGMDdictSoundFonts.pkl
!gsutil -m cp eGMDlabelsSoundFonts.pkl gs://fyr-audio-data/raw-audio/SoundFontFiles/eGMDlabelsSoundFonts.pkl
    
#os.remove('eGMDdict.pkl')
#os.remove('eGMDlabels.pkl')


Copying file://eGMDdict2.pkl [Content-Type=application/octet-stream]...
/ [1/1 files][555.1 KiB/555.1 KiB] 100% Done                                    
Operation completed over 1 objects/555.1 KiB.                                    
Copying file://eGMDlabels2.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1/1 files][2